In [136]:
import pymupdf
from pathlib import Path
import re
import os
from math import ceil

In [59]:
folder = Path('sample')

files = os.listdir(folder)


def extract_text(file_dir):

    doc = pymupdf.open(file_dir)

    text = ''

    for page in doc:
        text += page.get_text()

    return text

In [67]:
data = {}

for file in files:
    doc_text = extract_text(folder / file)

    data[file] = doc_text
    
    with open(f"{file}.txt", "w") as f:
        f.write(doc_text)

In [126]:
def insurer_name(text):
    companies = {
        "Reliance": "Reliance General Insurance Company",
        "ERGO": "HDFC ERGO General Insurance Company",
        "SBI": "SBI General Insurance Company",
        "New India": "India Assurance",
        "Future Generali": "Future Generali"
    }

    for company in companies:
        result = re.search(re.compile(companies[company], re.IGNORECASE), text)
        if result: return company

    return "NEW"

In [91]:
def policy_number(text):
    pattern1 = re.compile("policy number\s*:\s*([^\s\n]+)", re.IGNORECASE)
    pattern2 = re.compile("policy no.\s*([^\n]+)", re.IGNORECASE)
    
    result = re.search(pattern1, text) or re.search(pattern2, text)

    return result and result.group(1).replace(" ", "")

In [130]:
def vehical_number(text):

    pattern = re.compile("TN-?\d{2}-?[A-Z]{1,2}-?\d{4}", re.IGNORECASE)
    result = re.search(pattern, text)

    return (result and result.group().replace("-", "").upper()) or "NEW"

In [150]:
def customer_name(text):
    patterns = [
        re.compile("Mr.? ([a-zA-Z\.\s]+)"),
        re.compile("Ms.? ([^\n]+)"),
        re.compile("Insured'?s? Name\s*([^\n]+)")
    ]

    for pattern in patterns:
        result = re.search(pattern, text)
        if result: return (result and result.group(1))

for file in data:
    print(file, customer_name(data[file]))

reliance.pdf SANKAR  .
NO 
sbi.pdf PARATHAMAN M

india.pdf SURESH K
india.PDF GANESAMOORTHY  A
future.pdf RAJESH M
NO 
hdfc.PDF None


In [140]:
def premium(text):
    patterns = [
        re.compile("Total Payable.*\s*([\d.,]+)", re.IGNORECASE),
        re.compile("Total Premium.*\s*([\d.,]+)", re.IGNORECASE),
        re.compile("Total Premium \(rounded off\)\s*([\d.,]+)", re.IGNORECASE)
    ]

    for pattern in patterns:
        result = re.search(pattern, text)
        if result: return ceil(float(result.group(1).replace(",", ""))*(100/118))

for file in data:
    print(file, premium(data[file]))

In [127]:
file = 'test.pdf'

text = extract_text(folder/file)

In [145]:
customer_name(text)

'Conditions and Exclusions:'